## Imports

In [ ]:
import os
import numpy as np

import torch
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import CIFAR10, MNIST

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Model Settings

In [ ]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 28*28

# Other
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
GRAYSCALE = True

### Dataset

In [ ]:
transform = (
    transforms.Compose([
        #transforms.Resize(224),
        #transforms.RandomHorizontalFlip(p=.40),
        #transforms.RandomRotation(30),
        transforms.ToTensor(),
        #transforms.Normalize(
        #    (0.4914, 0.4822, 0.4465), 
        #    (0.2471, 0.2435, 0.2616)
        #)
    ])
)

In [ ]:
##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = MNIST(root='data', 
                      train=True, 
                      transform=transform, 
                      download=True)

test_dataset = MNIST(root='data', 
                     train=False, 
                     transform=transform)


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print(DEVICE)
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

cpu
Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


In [ ]:
device = torch.device(DEVICE)
torch.manual_seed(0)

for epoch in range(2):
    for batch_idx, (x, y) in enumerate(train_loader):
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 128
Epoch: 2 | Batch index: 0 | Batch size: 128


## Model

In [ ]:
from google.colab import drive


drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys


PATH = "/content/drive/My Drive/Colab Notebooks/EECS553ML_reproduce/"
#PATH = "/content/drive/My Drive/EECS553ML_reproduce/"
sys.path.append(PATH)

In [ ]:
#!cd "/content/drive/My Drive/EECS553ML_reproduce/";ls

In [ ]:
from resnet18 import ResNet152


torch.manual_seed(RANDOM_SEED)
model = ResNet152(size_for_cifar=False)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

## Training

In [ ]:
from trainer import Trainer


trainer = Trainer(model, train_loader, DEVICE)

Training on cpu


In [ ]:
trainer.train(LEARNING_RATE, NUM_EPOCHS)

Epoch: 001/010 | Batch 0000/0469 | Cost: 5.3727


KeyboardInterrupt: ignored

## Evaluation

In [ ]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (trainer.compute_accuracy(test_loader)))

In [ ]:
#%watermark -iv

In [ ]:
torch.save(model.state_dict(), f"{PATH}mnist_resnet152_model.t7")